### Start processing predicted and simulated stress-strain curves

#### Calculate predicted and simulated strength

In [ ]:
import numpy as np
import pandas as pd
import os

os.chdir('your_path_to_data') # change to your path to data
system=['SiO2','SiO2_C'] # the systems to be analyzed, can be extended to other more systems

voro_sys={} # store the voronoi volume data
ss={} # store the stress-strain data

n_data_for_prediction=6 # number of data for prediction

for sys in range(len(system)):
    ss[sys] = []
    voro_sys[sys] = []
    for i in range(n_data_for_prediction):
        voro = pd.read_csv('{}/voro_sio2_{}.csv'.format(system[sys],i), delimiter=',',header=None)
        ss_sio2 = pd.read_csv('{}/{}/loading/Re_Primary_Fracture.txt'.format(system[sys],i), sep=' ', header=None)
        ss[sys].append(ss_sio2)
        if system[sys]=='SiO2':
            type = 1 # Si, find well defined atoms for different systems
            v_sio2 = np.nanmean(voro[voro[0]==type].iloc[:,2:][voro[voro[0]==type].iloc[:,2:]<30],axis=0)
            voro_sys[sys].append(v_sio2)
        elif system[sys]=='SiO2_C':
            type = 1 # Si, find well defined atoms for different systems
            v_sio2 = np.nanmean(voro[voro[0]==type].iloc[:,2:][voro[voro[0]==type].iloc[:,2:]<30],axis=0)
            voro_sys[sys].append(v_sio2)

theory={} # store the predicted data
sigma={} # store the simulated data

k_sio2=7.9527523831363345 # volume constancy energy, taking SiO2 as an example. Can be calculated from linear regression for other systems

for j in range(len(system)):
    theory[j]=[]
    sigma[j]=[]
    theory_sio2_g=[]
    theory_sio2_c=[]
    sigma_sio2_g=[]
    sigma_sio2_c=[]
    for i in range(len(voro_sys[0])):
        if system[j]=='SiO2':
            theory_sio2_g.append((k_sio2/voro_sys[j][i][0]-k_sio2/voro_sys[j][i].max())*1.60217662*100)
            sigma_sio2_g.append(np.float64(ss[j][i].iloc[2:,5]).max())
        elif system[j]=='SiO2_C':
            theory_sio2_c.append((k_sio2/voro_sys[j][i][0]-k_sio2/voro_sys[j][i].max())*1.60217662*100)
            sigma_sio2_c.append(np.float64(ss[j][i].iloc[2:,5]).max())
    if system[j]=='SiO2':
        theory[j].append(theory_sio2_g)
        sigma[j].append(sigma_sio2_g)
    elif system[j]=='SiO2_C':
        theory[j].append(theory_sio2_c)
        sigma[j].append(sigma_sio2_c)

### Store data to pickle file

In [2]:
import pickle5 as pickle

with open('sigma_strength.pkl','wb') as f:
    pickle.dump(sigma,f)
with open('theory_strength.pkl','wb') as f:
    pickle.dump(theory,f)

#### Calculate predicted and simulated modulus of toughness, i.e., the area under stress-strain curve

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

os.chdir('your_path_to_data') # change to your path to data
system=['SiO2','SiO2_C'] # the systems to be analyzed, can be extended to other more systems
voro_sys={} # store the voronoi volume data
ss={} # store the stress-strain data

n_data_for_prediction=6 # number of data for prediction

for sys in range(len(system)):
    ss[sys] = []
    voro_sys[sys] = []
    for i in range(n_data_for_prediction):
        voro = pd.read_csv('{}/voro_sio2_{}.csv'.format(system[sys],i), delimiter=',',header=None)
        ss_sio2 = pd.read_csv('{}/{}/loading/Re_Primary_Fracture.txt'.format(system[sys],i), sep=' ', header=None)
        ss[sys].append(ss_sio2)
        if system[sys]=='SiO2':
            type = 1
            v_sio2 = np.nanmean(voro[voro[0]==type].iloc[:,2:][voro[voro[0]==type].iloc[:,2:]<30],axis=0)
            voro_sys[sys].append(v_sio2)
        elif system[sys]=='SiO2_C':
            type = 1
            v_sio2 = np.nanmean(voro[voro[0]==type].iloc[:,2:][voro[voro[0]==type].iloc[:,2:]<30],axis=0)
            voro_sys[sys].append(v_sio2)

k_sio2=7.9527523831363345 # volume constancy energy
strain = np.arange(0,1.001,0.01) # strain data

theory_toughness={} # store the predicted toughness data
sigma_toughness={} # store the simulated toughness data
ss_voro={}
for j in range(len(system)):
    theory[j]=[]
    sigma[j]=[]
    theory_sio2_g=[]
    theory_sio2_c=[]
    sigma_sio2_g=[]
    sigma_sio2_c=[]
    ss_voro[j]=[]
    for i in range(len(voro_sys[0])):
        if system[j]=='SiO2':
            v=np.array((k_sio2/voro_sys[j][i][0]-k_sio2/voro_sys[j][i])*1.60217662*100)
            ss_voro[j].append(v)
            index_of_fracture = defined_index # Consider only the area from the beginning of the stress-strain curve to the time of complete fracture
            theory_sio2_g.append(np.trapz(v[:index_of_fracture + 1],strain[:index_of_fracture + 1]))
            data = np.float64(ss[j][i].iloc[2:,5]) # simulated stress data
            x = np.float64(ss[j][i].iloc[2:,4]) # simulated strain data
            negative_index = np.where(data[10:] < 0)[0][0] + 10 # find the index of the first negative stress, which is the time of complete fracture
            sigma_sio2_g.append(np.trapz(data[:negative_index + 1],x[:negative_index + 1]))
        elif system[j]=='SiO2_C':
            v=np.array((k_sio2/voro_sys[j][i][0]-k_sio2/voro_sys[j][i])*1.60217662*100)
            ss_voro[j].append(v)
            index_of_fracture = defined_index # Find by different solid systems
            theory_sio2_c.append(np.trapz(v[:index_of_fracture + 1],strain[:index_of_fracture + 1]))
            data = np.float64(ss[j][i].iloc[2:,5])
            x = np.float64(ss[j][i].iloc[2:,4])
            negative_index = np.where(data[10:] < 0)[0][0] + 10
            sigma_sio2_c.append(np.trapz(data[:negative_index + 1],x[:negative_index + 1]))
    if system[j]=='SiO2':
        theory_toughness[j].append(theory_sio2_g)
        sigma_toughness[j].append(sigma_sio2_g)
    elif system[j]=='SiO2_C':
        theory_toughness[j].append(theory_sio2_c)
        sigma_toughness[j].append(sigma_sio2_c)

### Store data to pickle file

In [ ]:
import pickle5 as pickle

with open('sigma_toughness.pkl','wb') as f:
    pickle.dump(sigma_toughness,f)
with open('theory_toughness.pkl','wb') as f:
    pickle.dump(theory_toughness,f)